In [ ]:
from database import Database
from sklearn.pipeline import Pipeline
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

In [ ]:
db = Database()

get_hotel_mapping = db.get_hotel_mapping_as_df()
get_room_mapping = db.get_room_mapping_as_df()
get_meal_mapping = db.get_meal_mapping_as_df()
get_operator_mapping = db.get_operator_mapping_as_df()
reservation = db.reservations_booking

hotel_mapping = dict(
    zip(get_hotel_mapping["external_code"], get_hotel_mapping["hotel_id"])
)
room_mapping = dict(
    zip(
        get_room_mapping.apply(lambda x: (x["external_code"], x["hotel_id"]), axis=1),
        get_room_mapping["id"],
    )
)
meal_mapping = dict(zip(get_meal_mapping["code"], get_meal_mapping["id"]))

operator_mapping = dict(
    zip(get_operator_mapping["external_code"], get_operator_mapping["operator_id"])
)

In [ ]:
date_fr = "11/01/2022"
date_to = "10/31/2024"

In [ ]:
date_fr, date_to

In [ ]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        (
            "data_encoder",
            BookingDataEncoder(
                hotel_mapping,
                room_mapping,
                meal_mapping,
                operator_mapping,
            ),
        ),
    ]
)

In [ ]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 2)
    get_bookings = booking_data.get()

    if get_bookings is not None:
        df = pipeline.fit_transform(get_bookings)

    dfs.append(df)

In [ ]:
bookings = pd.concat(dfs, axis=0, ignore_index=True)
bookings.to_csv("bookings.csv", index=False)

In [ ]:
import pandas as pd

In [ ]:
bookings.isna().sum()